# ICPR 2024 Competition on Multilingual Claim-Span Identification

## Installing Dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!pip install --upgrade scikit-learn
#!pip install --upgrade imbalanced-learn

### Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV

### Loading Datasets

In [ ]:
# Training English and Hindi datasets
train_data = pd.read_json("/content/drive/MyDrive/Multilingual Datasets/ML Data/train_en_hi_encoded_labels.json")

# Validation English dataset
val_data_en = pd.read_json("/content/drive/MyDrive/Multilingual Datasets/Clean Data/val_en_encoded_clean.json")

# Validation Hindi dataset
val_data_hi = pd.read_json("/content/drive/MyDrive/Multilingual Datasets/Clean Data/val_hi_encoded_clean.json")

#Test English dataset
test_data_en = pd.read_json("/content/drive/MyDrive/Multilingual Datasets/Test Data/Test Data Clean/test_en_clean.json")

#Test Hindi dataset
test_data_hi = pd.read_json("/content/drive/MyDrive/Multilingual Datasets/Test Data/Test Data Clean/test_hi_clean.json")

In [ ]:
test_data_en.head()

,text_tokens
0,@u20719141 @u55524931 we have to assume no vac...
1,@u41101034 same as my union screaming out for ...
2,this is why i laff talked all that shit went a...
3,@u21887764 ugh that is tragic i have a cousin ...
4,@u40601807 @u69515872 heroes the direction tho...


In [ ]:
test_data_hi.head()

,text_tokens
0,एक नचनिया के अवैध निर्माण पर कार्रवाई से बीजेप...
1,अब चुमार में घुसपैठ की कोशिश नाकाम भारत बोला स...
2,RT @ U64277340 ड्रग्स भी लेती थीं बीफ भी खाती ...
3,अनलॉक 4 में रेलवे चला सकता है 100 नई ट्रेनें इ...
4,अभी तक आरोपी लाखों लीटर ऑयल चुरा चुके हैं @ U1...


In [ ]:
val_data_en.head()

,claims,text_tokens
0,1,listen people the vaccines work they work that...
1,0,i ll take the covid vaccine in 5 years after a...
2,0,@u72240666 hearing that the pope says refusing...
3,1,trump has got the new russian vaccine and he c...
4,1,@u41390182 @u31519664 gotta agree we already i...


In [ ]:
train_data.head()

,claims,text_tokens,labels
0,0,#vaers 17y #pfizer #covidvaccine #suicide atte...,not_claim
1,1,weve truly come a long way from december and j...,claim
2,1,fuck that its not faux outrage inject them wit...,claim
3,1,@u55750420 which makes no sense the vaccine ca...,claim
4,1,fact or fiction you decide the upcoming corona...,claim


In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12068 entries, 0 to 12067
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   claims       12068 non-null  int64 
 1   text_tokens  12068 non-null  object
 2   labels       12068 non-null  object
dtypes: int64(1), object(2)
memory usage: 283.0+ KB


In [ ]:
val_data_en.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499 entries, 0 to 498
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   claims       499 non-null    int64 
 1   text_tokens  499 non-null    object
dtypes: int64(1), object(1)
memory usage: 7.9+ KB


### Feature Extraction

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Define pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer(use_idf=True))
])

# Apply pipeline on training data
X = pipeline.fit_transform(train_data['text_tokens'])
y = train_data['claims']

# Apply same pipeline on validation data
#X_val = pipeline.transform(val_data['text_tokens'])
#y_val = val_data['claims']


# Apply same pipeline on English validation data
X_val_en = pipeline.transform(val_data_en['text_tokens'])
y_val_en = val_data_en['claims']


# Apply same pipeline on Hindi validation data
X_val_hi = pipeline.transform(val_data_hi['text_tokens'])
y_val_hi = val_data_hi['claims']

# Apply same pipeline on English Test data
X_test_en = pipeline.transform(test_data_en['text_tokens'])
#y_test_en = val_data_en['claims']

# Apply same pipeline on Hindi Test data
X_test_hi = pipeline.transform(test_data_hi['text_tokens'])
#y_test_en = val_data_en['claims']


print("----Training Data------")
print(X.shape)
print(y.shape)

print("\n----English Validation Data------")
print(X_val_en.shape)
print(y_val_en.shape)

print("\n----Hindi Validation Data------")
print(X_val_hi.shape)
print(y_val_hi.shape)

print("\n----English test Data------")
print(test_data_en.shape)
#print(y_val_en.shape)

print("\n----Hindi test Data------")
print(test_data_hi.shape)
#print(y_val_en.shape)

----Training Data------
(12068, 24811)
(12068,)

----English Validation Data------
(499, 24811)
(499,)

----Hindi Validation Data------
(500, 24811)
(500,)

----English test Data------
(1500, 1)

----Hindi test Data------
(1500, 1)


In [ ]:
# Splitting the dataset into the Training set and testing Set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [ ]:
y_train.value_counts()

claims
1    8830
0    2031
Name: count, dtype: int64

__After checking the training set, there are imbalance of data and it may cause a bias in training the model. Oversampling using `SMOTE` will be used to balance the train dataset.__

### SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter

# Check the initial class distribution
print("Initial class distribution:", Counter(y_train))

Initial class distribution: Counter({1: 8830, 0: 2031})


In [ ]:
# Initialize SMOTE with a strategy to balance all classes equally
smote = SMOTE(sampling_strategy='auto', random_state=42)


# Apply SMOTE
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

In [ ]:
# Check the class distribution after SMOTE
print("Resampled class distribution:", Counter(y_train_res))

Resampled class distribution: Counter({1: 8830, 0: 8830})


__Now, the training set is balanced and ready to be feed into the ML models.__

### Model Training & Development

### Decision Trees Classifier

In [ ]:
from sklearn.metrics import jaccard_score, f1_score
from sklearn.svm import SVC

In [ ]:
# Decision Trees

from sklearn.tree import DecisionTreeClassifier
import sklearn.tree as tree

Tree_clf = DecisionTreeClassifier(criterion="entropy", max_depth=42, random_state=42)  #80
Tree_clf.fit(X_train, y_train)

tree_pred = Tree_clf.predict(X_test_en)
tree_pred_hi = Tree_clf.predict(X_test_hi)

test_data_en["target"] = Tree_clf.predict(X_test_en)
test_data_hi["target"] = Tree_clf.predict(X_test_hi)

In [ ]:
claims = ['not_claim', 'claim']

print('Classification Report for Decision Trees using English Test dataset:\n',classification_report(test_data_en.target, tree_pred, target_names=claims))

Classification Report for Decision Trees using English Test dataset:
               precision    recall  f1-score   support

   not_claim       1.00      1.00      1.00       244
       claim       1.00      1.00      1.00      1256

    accuracy                           1.00      1500
   macro avg       1.00      1.00      1.00      1500
weighted avg       1.00      1.00      1.00      1500



In [ ]:
print('Classification Report for Decision Trees using Hindi Test dataset:\n',classification_report(test_data_hi.target, tree_pred_hi, target_names=claims))

Classification Report for Decision Trees using Hindi Test dataset:
               precision    recall  f1-score   support

   not_claim       1.00      1.00      1.00       159
       claim       1.00      1.00      1.00      1341

    accuracy                           1.00      1500
   macro avg       1.00      1.00      1.00      1500
weighted avg       1.00      1.00      1.00      1500



In [ ]:
from sklearn.metrics import f1_score, jaccard_score
import numpy as np

# Step 1: Make predictions on the validation set
predictions = (Tree_clf.predict(X_test_en) > 0.5).astype(int)

# Initialize lists to hold individual scores
jaccard_scores = []
macro_f1_scores = []

# Iterate over each post
for i in range(len(test_data_en.target)):
    # Ensure y_val[i] and predictions[i] are arrays
    y_true_post = np.array(test_data_en.target[i]).reshape(1, -1)
    y_pred_post = np.array(predictions[i]).reshape(1, -1)

    # Calculate Jaccard similarity score for each post
    jaccard = jaccard_score(y_true_post[0], y_pred_post[0], zero_division=0)
    jaccard_scores.append(jaccard)

    # Calculate Macro-F1 score for each post
    macro_f1 = f1_score(y_true_post[0], y_pred_post[0], average='macro', zero_division=0)
    macro_f1_scores.append(macro_f1)

# Calculate the average of the scores
average_jaccard = np.mean(jaccard_scores)
average_macro_f1 = np.mean(macro_f1_scores)

print("Average Jaccard Similarity Score of Decision Trees using English Test data:", average_jaccard)
print("Average Macro-F1 Score of Decision Trees using English Test data:", average_macro_f1)

Average Jaccard Similarity Score of Decision Trees using English Test data: 0.8373333333333334
Average Macro-F1 Score of Decision Trees using English Test data: 1.0


In [ ]:
from sklearn.metrics import f1_score, jaccard_score
import numpy as np

# Step 1: Make predictions on the validation set
predictions = (Tree_clf.predict(X_test_hi) > 0.5).astype(int)

# Initialize lists to hold individual scores
jaccard_scores = []
macro_f1_scores = []

# Iterate over each post
for i in range(len(test_data_hi.target)):
    # Ensure y_val[i] and predictions[i] are arrays
    y_true_post = np.array(test_data_hi.target[i]).reshape(1, -1)
    y_pred_post = np.array(predictions[i]).reshape(1, -1)

    # Calculate Jaccard similarity score for each post
    jaccard = jaccard_score(y_true_post[0], y_pred_post[0], zero_division=0)
    jaccard_scores.append(jaccard)

    # Calculate Macro-F1 score for each post
    macro_f1 = f1_score(y_true_post[0], y_pred_post[0], average='macro', zero_division=0)
    macro_f1_scores.append(macro_f1)

# Calculate the average of the scores
average_jaccard = np.mean(jaccard_scores)
average_macro_f1 = np.mean(macro_f1_scores)

print("Average Jaccard Similarity Score of Decision Trees using Hindi Test data:", average_jaccard)
print("Average Macro-F1 Score of Decision Trees using Hindi Test data:", average_macro_f1)

Average Jaccard Similarity Score of Decision Trees using Hindi Test data: 0.894
Average Macro-F1 Score of Decision Trees using Hindi Test data: 1.0


### Logistic Regression Model

In [ ]:
#Logistic Regression Model

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# train a Logistic Regression Model
LR_clf = LogisticRegression(max_iter = 1000) #83
LR_clf.fit(X_train, y_train)

# evaluate the classifier on the test set
LR_pred = LR_clf.predict(X_test_en)
LR_pred_hi = LR_clf.predict(X_test_hi)

test_data_en["target1"] = LR_clf.predict(X_test_en)
test_data_hi["target1"] = LR_clf.predict(X_test_hi)

In [ ]:
print('Classification Report for Logistic Regression using English Test dataset :\n',classification_report(test_data_en.target1, LR_pred, target_names=claims))

Classification Report for Logistic Regression using English Test dataset :
               precision    recall  f1-score   support

   not_claim       1.00      1.00      1.00        52
       claim       1.00      1.00      1.00      1448

    accuracy                           1.00      1500
   macro avg       1.00      1.00      1.00      1500
weighted avg       1.00      1.00      1.00      1500



In [ ]:
print('Classification Report for Logistic Regression using Hindi Test dataset :\n',classification_report(test_data_hi.target1, LR_pred_hi, target_names=claims))

Classification Report for Logistic Regression using Hindi Test dataset :
               precision    recall  f1-score   support

   not_claim       1.00      1.00      1.00        72
       claim       1.00      1.00      1.00      1428

    accuracy                           1.00      1500
   macro avg       1.00      1.00      1.00      1500
weighted avg       1.00      1.00      1.00      1500



In [ ]:
from sklearn.metrics import f1_score, jaccard_score
import numpy as np

# Step 1: Make predictions on the validation set
predictions = (LR_clf.predict(X_test_en) > 0.5).astype(int)

# Initialize lists to hold individual scores
jaccard_scores = []
macro_f1_scores = []

# Iterate over each post
for i in range(len(test_data_en.target1)):
    # Ensure y_val[i] and predictions[i] are arrays
    y_true_post = np.array(test_data_en.target1[i]).reshape(1, -1)
    y_pred_post = np.array(predictions[i]).reshape(1, -1)

    # Calculate Jaccard similarity score for each post
    jaccard = jaccard_score(y_true_post[0], y_pred_post[0], zero_division=0)
    jaccard_scores.append(jaccard)

    # Calculate Macro-F1 score for each post
    macro_f1 = f1_score(y_true_post[0], y_pred_post[0], average='macro', zero_division=0)
    macro_f1_scores.append(macro_f1)

# Calculate the average of the scores
average_jaccard = np.mean(jaccard_scores)
average_macro_f1 = np.mean(macro_f1_scores)

print("Average Jaccard Similarity Score of Logistic Regression using English Test data:", average_jaccard)
print("Average Macro-F1 Score of Logistic Regression using English Test data:", average_macro_f1)

Average Jaccard Similarity Score of Logistic Regression using English Test data: 0.9653333333333334
Average Macro-F1 Score of Logistic Regression using English Test data: 1.0


In [ ]:
from sklearn.metrics import f1_score, jaccard_score
import numpy as np

# Step 1: Make predictions on the validation set
predictions = (LR_clf.predict(X_test_hi) > 0.5).astype(int)

# Initialize lists to hold individual scores
jaccard_scores = []
macro_f1_scores = []

# Iterate over each post
for i in range(len(test_data_hi.target1)):
    # Ensure y_val[i] and predictions[i] are arrays
    y_true_post = np.array(test_data_hi.target1[i]).reshape(1, -1)
    y_pred_post = np.array(predictions[i]).reshape(1, -1)

    # Calculate Jaccard similarity score for each post
    jaccard = jaccard_score(y_true_post[0], y_pred_post[0], zero_division=0)
    jaccard_scores.append(jaccard)

    # Calculate Macro-F1 score for each post
    macro_f1 = f1_score(y_true_post[0], y_pred_post[0], average='macro', zero_division=0)
    macro_f1_scores.append(macro_f1)

# Calculate the average of the scores
average_jaccard = np.mean(jaccard_scores)
average_macro_f1 = np.mean(macro_f1_scores)

print("Average Jaccard Similarity Score of Logistic Regression using Hindi test data:", average_jaccard)
print("Average Macro-F1 Score of Logistic Regression using Hindi test data:", average_macro_f1)

Average Jaccard Similarity Score of Logistic Regression using Hindi test data: 0.952
Average Macro-F1 Score of Logistic Regression using Hindi test data: 1.0


### SVM (Support Vector Machines)

In [ ]:
# SVM (Support Vector Machines)

from sklearn import svm

SV_clf = svm.SVC(kernel='rbf')  #83
SV_clf.fit(X_train, y_train)

svm_pred = SV_clf.predict(X_test_en)
svm_pred_hi = SV_clf.predict(X_test_hi)


test_data_en["target2"] = SV_clf.predict(X_test_en)
test_data_hi["target2"] = SV_clf.predict(X_test_hi)

In [ ]:
print('Classification Report for Support Vector Machines using English test dataset:\n',classification_report(test_data_en.target2, svm_pred, target_names=claims))

Classification Report for Support Vector Machines using English test dataset:
               precision    recall  f1-score   support

   not_claim       1.00      1.00      1.00        35
       claim       1.00      1.00      1.00      1465

    accuracy                           1.00      1500
   macro avg       1.00      1.00      1.00      1500
weighted avg       1.00      1.00      1.00      1500



In [ ]:
print('Classification Report for Support Vector Machines using Hindi test dataset:\n',classification_report(test_data_hi.target2, svm_pred_hi, target_names=claims))

Classification Report for Support Vector Machines using Hindi test dataset:
               precision    recall  f1-score   support

   not_claim       1.00      1.00      1.00        58
       claim       1.00      1.00      1.00      1442

    accuracy                           1.00      1500
   macro avg       1.00      1.00      1.00      1500
weighted avg       1.00      1.00      1.00      1500



In [ ]:
from sklearn.metrics import f1_score, jaccard_score
import numpy as np

# Step 1: Make predictions on the validation set
predictions = (SV_clf.predict(X_test_en) > 0.5).astype(int)

# Initialize lists to hold individual scores
jaccard_scores = []
macro_f1_scores = []

# Iterate over each post
for i in range(len(test_data_en.target2)):
    # Ensure y_val[i] and predictions[i] are arrays
    y_true_post = np.array(test_data_en.target2[i]).reshape(1, -1)
    y_pred_post = np.array(predictions[i]).reshape(1, -1)

    # Calculate Jaccard similarity score for each post
    jaccard = jaccard_score(y_true_post[0], y_pred_post[0], zero_division=0)
    jaccard_scores.append(jaccard)

    # Calculate Macro-F1 score for each post
    macro_f1 = f1_score(y_true_post[0], y_pred_post[0], average='macro', zero_division=0)
    macro_f1_scores.append(macro_f1)

# Calculate the average of the scores
average_jaccard = np.mean(jaccard_scores)
average_macro_f1 = np.mean(macro_f1_scores)

print("Average Jaccard Similarity Score of SVM using English test data:", average_jaccard)
print("Average Macro-F1 Score of SVM using English test data:", average_macro_f1)

Average Jaccard Similarity Score of SVM using English test data: 0.9766666666666667
Average Macro-F1 Score of SVM using English test data: 1.0


In [ ]:
from sklearn.metrics import f1_score, jaccard_score
import numpy as np

# Step 1: Make predictions on the validation set
predictions = (SV_clf.predict(X_test_hi) > 0.5).astype(int)

# Initialize lists to hold individual scores
jaccard_scores = []
macro_f1_scores = []

# Iterate over each post
for i in range(len(test_data_hi.target2)):
    # Ensure y_val[i] and predictions[i] are arrays
    y_true_post = np.array(test_data_hi.target2[i]).reshape(1, -1)
    y_pred_post = np.array(predictions[i]).reshape(1, -1)

    # Calculate Jaccard similarity score for each post
    jaccard = jaccard_score(y_true_post[0], y_pred_post[0], zero_division=0)
    jaccard_scores.append(jaccard)

    # Calculate Macro-F1 score for each post
    macro_f1 = f1_score(y_true_post[0], y_pred_post[0], average='macro', zero_division=0)
    macro_f1_scores.append(macro_f1)

# Calculate the average of the scores
average_jaccard = np.mean(jaccard_scores)
average_macro_f1 = np.mean(macro_f1_scores)

print("Average Jaccard Similarity Score of SVM using Hindi test data:", average_jaccard)
print("Average Macro-F1 Score of SVM using Hindi test data:", average_macro_f1)

Average Jaccard Similarity Score of SVM using Hindi test data: 0.9613333333333334
Average Macro-F1 Score of SVM using Hindi test data: 1.0


### RandomForest Classifier

In [ ]:
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)

rf_pred = rf_clf.predict(X_test_en)
rf_pred_hi = rf_clf.predict(X_test_hi)

test_data_en["target3"] = rf_clf.predict(X_test_en)
test_data_hi["target3"] = rf_clf.predict(X_test_hi)

In [ ]:
print('Classification Report for Random Forest using English test dataset:\n',classification_report(test_data_en.target3, rf_pred, target_names=claims))

Classification Report for Random Forest using English test dataset:
               precision    recall  f1-score   support

   not_claim       1.00      1.00      1.00        45
       claim       1.00      1.00      1.00      1455

    accuracy                           1.00      1500
   macro avg       1.00      1.00      1.00      1500
weighted avg       1.00      1.00      1.00      1500



In [ ]:
print('Classification Report for Random Forest using Hindi test dataset:\n',classification_report(test_data_hi.target3, rf_pred_hi, target_names=claims))

Classification Report for Random Forest using Hindi test dataset:
               precision    recall  f1-score   support

   not_claim       1.00      1.00      1.00       118
       claim       1.00      1.00      1.00      1382

    accuracy                           1.00      1500
   macro avg       1.00      1.00      1.00      1500
weighted avg       1.00      1.00      1.00      1500



In [ ]:
from sklearn.metrics import f1_score, jaccard_score
import numpy as np

# Step 1: Make predictions on the validation set
predictions = (rf_clf.predict(X_test_en) > 0.5).astype(int)

# Initialize lists to hold individual scores
jaccard_scores = []
macro_f1_scores = []

# Iterate over each post
for i in range(len(test_data_en.target3)):
    # Ensure y_val[i] and predictions[i] are arrays
    y_true_post = np.array(test_data_en.target3[i]).reshape(1, -1)
    y_pred_post = np.array(predictions[i]).reshape(1, -1)

    # Calculate Jaccard similarity score for each post
    jaccard = jaccard_score(y_true_post[0], y_pred_post[0], zero_division=0)
    jaccard_scores.append(jaccard)

    # Calculate Macro-F1 score for each post
    macro_f1 = f1_score(y_true_post[0], y_pred_post[0], average='macro', zero_division=0)
    macro_f1_scores.append(macro_f1)

# Calculate the average of the scores
average_jaccard = np.mean(jaccard_scores)
average_macro_f1 = np.mean(macro_f1_scores)

print("Average Jaccard Similarity Score of Random Forest using English test data:", average_jaccard)
print("Average Macro-F1 Score of Random Forest using English test data:", average_macro_f1)

Average Jaccard Similarity Score of Random Forest using English test data: 0.97
Average Macro-F1 Score of Random Forest using English test data: 1.0


In [ ]:
from sklearn.metrics import f1_score, jaccard_score
import numpy as np

# Step 1: Make predictions on the validation set
predictions = (rf_clf.predict(X_test_hi) > 0.5).astype(int)

# Initialize lists to hold individual scores
jaccard_scores = []
macro_f1_scores = []

# Iterate over each post
for i in range(len(test_data_hi.target3)):
    # Ensure y_val[i] and predictions[i] are arrays
    y_true_post = np.array(test_data_hi.target3[i]).reshape(1, -1)
    y_pred_post = np.array(predictions[i]).reshape(1, -1)

    # Calculate Jaccard similarity score for each post
    jaccard = jaccard_score(y_true_post[0], y_pred_post[0], zero_division=0)
    jaccard_scores.append(jaccard)

    # Calculate Macro-F1 score for each post
    macro_f1 = f1_score(y_true_post[0], y_pred_post[0], average='macro', zero_division=0)
    macro_f1_scores.append(macro_f1)

# Calculate the average of the scores
average_jaccard = np.mean(jaccard_scores)
average_macro_f1 = np.mean(macro_f1_scores)

print("Average Jaccard Similarity Score of Random Forest using Hindi test data:", average_jaccard)
print("Average Macro-F1 Score of Random Forest using Hindi test data:", average_macro_f1)

Average Jaccard Similarity Score of Random Forest using Hindi test data: 0.9213333333333333
Average Macro-F1 Score of Random Forest using Hindi test data: 1.0


### XGBoost Classifier algorithm

In [ ]:
from xgboost import XGBClassifier

xgboost = XGBClassifier()
xgboost.fit(X_train, y_train)

xgboost_pred = xgboost.predict(X_test_en)
xgboost_pred_hi = xgboost.predict(X_test_hi)

test_data_en["target5"] = xgboost.predict(X_test_en)
test_data_hi["target5"] = xgboost.predict(X_test_hi)

In [ ]:
print('Classification Report for XGBoost Classifier using English test dataset:\n',classification_report(test_data_en.target5, xgboost_pred, target_names=claims))

Classification Report for XGBoost Classifier using English test dataset:
               precision    recall  f1-score   support

   not_claim       1.00      1.00      1.00       110
       claim       1.00      1.00      1.00      1390

    accuracy                           1.00      1500
   macro avg       1.00      1.00      1.00      1500
weighted avg       1.00      1.00      1.00      1500



In [ ]:
print('Classification Report for XGBoost Classifier using Hindi test dataset:\n',classification_report(test_data_hi.target5, xgboost_pred_hi, target_names=claims))

Classification Report for XGBoost Classifier using Hindi test dataset:
               precision    recall  f1-score   support

   not_claim       1.00      1.00      1.00       100
       claim       1.00      1.00      1.00      1400

    accuracy                           1.00      1500
   macro avg       1.00      1.00      1.00      1500
weighted avg       1.00      1.00      1.00      1500



In [ ]:
from sklearn.metrics import f1_score, jaccard_score
import numpy as np

# Step 1: Make predictions on the validation set
predictions = (xgboost.predict(X_test_en) > 0.5).astype(int)

# Initialize lists to hold individual scores
jaccard_scores = []
macro_f1_scores = []

# Iterate over each post
for i in range(len(test_data_en.target5)):
    # Ensure y_val[i] and predictions[i] are arrays
    y_true_post = np.array(test_data_en.target5[i]).reshape(1, -1)
    y_pred_post = np.array(predictions[i]).reshape(1, -1)

    # Calculate Jaccard similarity score for each post
    jaccard = jaccard_score(y_true_post[0], y_pred_post[0], zero_division=0)
    jaccard_scores.append(jaccard)

    # Calculate Macro-F1 score for each post
    macro_f1 = f1_score(y_true_post[0], y_pred_post[0], average='macro', zero_division=0)
    macro_f1_scores.append(macro_f1)

# Calculate the average of the scores
average_jaccard = np.mean(jaccard_scores)
average_macro_f1 = np.mean(macro_f1_scores)

print("Average Jaccard Similarity Score of XGBoost using English test data:", average_jaccard)
print("Average Macro-F1 Score of XGBoost using English test data:", average_macro_f1)

Average Jaccard Similarity Score of XGBoost using English test data: 0.9266666666666666
Average Macro-F1 Score of XGBoost using English test data: 1.0


In [ ]:
from sklearn.metrics import f1_score, jaccard_score
import numpy as np

# Step 1: Make predictions on the validation set
predictions = (xgboost.predict(X_test_hi) > 0.5).astype(int)

# Initialize lists to hold individual scores
jaccard_scores = []
macro_f1_scores = []

# Iterate over each post
for i in range(len(test_data_hi.target5)):
    # Ensure y_val[i] and predictions[i] are arrays
    y_true_post = np.array(test_data_hi.target5[i]).reshape(1, -1)
    y_pred_post = np.array(predictions[i]).reshape(1, -1)

    # Calculate Jaccard similarity score for each post
    jaccard = jaccard_score(y_true_post[0], y_pred_post[0], zero_division=0)
    jaccard_scores.append(jaccard)

    # Calculate Macro-F1 score for each post
    macro_f1 = f1_score(y_true_post[0], y_pred_post[0], average='macro', zero_division=0)
    macro_f1_scores.append(macro_f1)

# Calculate the average of the scores
average_jaccard = np.mean(jaccard_scores)
average_macro_f1 = np.mean(macro_f1_scores)

print("Average Jaccard Similarity Score of XGBoost using Hindi test data:", average_jaccard)
print("Average Macro-F1 Score of XGBoost using Hindi test data:", average_macro_f1)

Average Jaccard Similarity Score of XGBoost using Hindi test data: 0.9333333333333333
Average Macro-F1 Score of XGBoost using Hindi test data: 1.0
